In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.experimental.output_all_intermediates(True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

import keras.backend.tensorflow_backend as tfback
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings)

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()

import keras
from keras import layers
from keras.optimizers import Adam
keras.backend.clear_session()

In [ ]:
def get_model(img_size, num_classes):
    inputs = layers.Input(shape=img_size)

    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual]) 
        previous_block_activation = x  

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])
        previous_block_activation = x 

    outputs = layers.Conv2D(num_classes, 3, activation="sigmoid", padding="same")(x)

    model = keras.Model(inputs, outputs)
    return model

keras.backend.clear_session()
model = get_model((512 , 768 , 3), 1)
model.summary()

In [ ]:
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.load_weights('unet_weights.h5')

In [ ]:
img = cv2.imread('img.jpg')
img = cv2.resize(img , (768,512))
mask = cv2.imread('mask.jpg', 0)
mask = cv2.resize(mask , (768,512))
mask = np.array(mask == 255 , np.int32)
mask = np.expand_dims(mask , 2)
img = np.expand_dims(img , 0)/255.0
mask = np.expand_dims(mask , 0)

In [ ]:
img.shape

In [ ]:
model.fit(img , mask , epochs=10)

In [ ]:
model.save_weights('unet_weights1.h5')

In [ ]:
from keras.models import model_from_json
json_model = model.to_json()
with open('mask.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import keras.backend.tensorflow_backend as tfback
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings)

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()
tf.config.list_logical_devices()
import keras
keras.backend.clear_session()

In [ ]:
from keras.models import model_from_json
with open('mask.json', 'r') as json_file:
    json_savedModel= json_file.read()
model = model_from_json(json_savedModel)
model.summary()

In [ ]:
model.load_weights('unet_weights.h5')
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [ ]:
def test(no , prob):
    img_path = 'Dataset/damaged/' + str(no)+'.jpg'
    mask_path = 'Dataset/masks/' + str(no)+'.jpg'
    img = cv2.imread(img_path)
    maskr = cv2.imread(mask_path)
    mask = model.predict(np.expand_dims(img , 0)/255.0)
    mask = (mask > prob) *255.0
    mask = np.reshape(mask , (mask.shape[1] , mask.shape[2])).astype('uint8')
    mask = cv2.cvtColor(mask , cv2.COLOR_GRAY2BGR)
    plt.imshow(cv2.cvtColor(img , cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(maskr)
    plt.show()
    plt.imshow(mask)
    plt.show()

In [ ]:
test(656 , 0.3)